In [ ]:
# -*- coding: utf-8 -*-
"""
Carga diaria de SalesNapse_<fecha>.csv  →  stage_napse (SQL Server)
Harney Márquez · 2025-06-26
"""
import pandas as pd
import pyodbc
import shutil
import sys
from pathlib import Path
from datetime import datetime
from datetime import timedelta
# ----------------------------------------------------------------------
# 1.  Parámetros generales
# ----------------------------------------------------------------------
BASE_PATH   = Path(r"\\192.168.10.5\ETLs_input\BI\NAPSE\Ventas")
DESTINO_OK = Path(r"\\192.168.10.5\ETLs_input\BI\NAPSE\Ventas\Insertados_reingenieria")/ "Insertados_reingenieria"

FECHA_ARCH  = datetime.now().strftime("%Y-%m-%d")
FILE_PATH   = BASE_PATH / f"SalesNapse_{FECHA_ARCH}.csv"

SQL_DRIVER  = "ODBC Driver 18 for SQL Server"
SQL_SERVER  = r"SRVSQLBIDEV\SQLSERVERDEV"
SQL_DB      = "MOBODW_STG"
SQL_USER    = "SA"

TABLE_NAME  = "dbo.stage_napse"
BATCH_SIZE  = 5000

# ----------------------------------------------------------------------
# 2.  Verifica existencia del archivo
# ----------------------------------------------------------------------
# 2. Verifica existencia del archivo
if not FILE_PATH.exists():
    print(f"❌ No se encontró el archivo: {FILE_PATH}")

    # Enviar correo de alerta
    from email.message import EmailMessage
    import smtplib

    SERVER_HOST_MAIL = "smtp.sendgrid.net"
    SERVER_PORT_MAIL = 587
    SERVER_USER_MAIL = "apikey"
    SERVER_FROM_MAIL = "no-reply@mobonet.mx"
    DESTINATARIOS = ["jmarquez@mobo.com.mx"]

    def enviar_correo(asunto, cuerpo):
        msg = EmailMessage()
        msg["Subject"] = asunto
        msg["From"] = SERVER_FROM_MAIL
        msg["To"] = ", ".join(DESTINATARIOS)
        msg.set_content(cuerpo)
        try:
            with smtplib.SMTP(SERVER_HOST_MAIL, SERVER_PORT_MAIL) as smtp:
                smtp.starttls()
                smtp.login(SERVER_USER_MAIL, SERVER_PASS_MAIL)
                smtp.send_message(msg)
            print("📧 Correo de alerta enviado.")
        except Exception as e:
            print("❌ Error al enviar correo de alerta:", e)

    enviar_correo(
        asunto="⚠️ Alerta: Archivo Napse no encontrado",
        cuerpo=f"No se encontró el archivo esperado para hoy:\n\n{FILE_PATH}\n\nNo se realizó ninguna carga ni eliminación en la base de datos."
    )

    sys.exit()  # Detener el script sin continuar
        # Detener el script sin continuar


if FILE_PATH.exists():
    # Calcula fechas dinámicas para los últimos 10 días
    fecha_fin = datetime.now() - timedelta(days=1)
    fecha_ini = fecha_fin - timedelta(days=10)
    
    fecha_ini_str = fecha_ini.strftime("%Y%m%d")
    fecha_fin_str = fecha_fin.strftime("%Y%m%d")

    print(f"🧹 Eliminando registros de stage_napse entre {fecha_ini_str} y {fecha_fin_str}...")

    try:
        with pyodbc.connect(
            f"DRIVER={{{SQL_DRIVER}}};SERVER={SQL_SERVER};DATABASE={SQL_DB};UID={SQL_USER};PWD={SQL_PASS};Encrypt=yes;TrustServerCertificate=yes"
        ) as conn:
            cursor = conn.cursor()
            delete_sql = f"""
                DELETE FROM {TABLE_NAME}
                WHERE fecha BETWEEN ? AND ?
            """
            cursor.execute(delete_sql, (fecha_ini_str, fecha_fin_str))
            conn.commit()
            print("✅ Registros eliminados exitosamente")
    except Exception as e:
        print(f"⚠️  Error eliminando datos previos: {e}")



# ----------------------------------------------------------------------
# 3.  Lee el CSV
# ----------------------------------------------------------------------
df = pd.read_csv(
    FILE_PATH,
    sep=",",
    encoding="utf-8-sig",
    dtype=str,
    keep_default_na=False
)

# ----------------------------------------------------------------------
# 4.  Mapeo de columnas
# ----------------------------------------------------------------------
column_map = {
    "venta_sk": "venta_sk", "venta_sk_n": "venta_sk_n", "fecha": "ffecha", "hora": "hora",
    "idcliente": "cliente.telefono", "sucursal": "sucursal", "caja": "caja", "idempleado": "items_.sellerID",
    "almacen": "almacen", "sku": "items_.SKU", "cantidad": "items_.quantity",
    "precio_s_impuestos": "items_.priceWoVAT", "impuesto": "items_.IVA", "subtotal": "subtotal",
    "descuento": "items_.total_disc", "precioneto_c_desc": None, "importe_c_desc": "items_.importe_c_desc",
    "total_venta": "total", "monto_pagado": "formas_pago.fp_amount", "forma_pago": "formas_pago.fopa",
    "tipo": "tipo_venta", "num_linea": "numLine", "canceled": None, "DiscPrcnt": None, "canal": None,
    "subcanal": None, "descuento_cupon": None, "id_cupon_gen": None, "conf_cupon_gen": None,
    "estatus_cupon_gen": None, "id_cupon_red": None, "conf_cupon_red": None, "codigo_promo": None,
    "porc_descuento_nm": None, "cant_gratis_nm": None, "datos_promo": None, "fecha_process": "fecha_process",
    "estatus": None, "inre": None, "folio_apart_liq": None, "forma_trans": None,
    "referencia": "folio_vinculado", "linea_pago": "formas_pago.fp_installments", "fecha_pago": "formas_pago.fp_date",
    "id_canal": None, "id_tmp": None, "suc_pago": None, "ref_pago": None, "autorizacion": "formas_pago.fp_auth_code",
    "origen": "origin", "num_line": None, "flag": None, "precio_bruto_s_desc": "items_.gross_up",
    "installments": "formas_pago.fp_installments", "fopa": None, "nom_cliente": "cliente.nombre",
    "gen_cliente": "cliente.genero", "price_list": "items_.price_list", "nom_lista": "items_.price_list",
    "hora_round": "hora", "email_cliente": "emailCustomer", "name_cliente": "nameCustomer",
    "estatus_sku": None, "folio_vinculado": "folio_vinculado"
}

ordered_cols = list(column_map.keys())
df_sql = pd.DataFrame(columns=ordered_cols)

for dest_col, src_col in column_map.items():
    if src_col is None:
        df_sql[dest_col] = None
    elif src_col in df.columns:
        df_sql[dest_col] = df[src_col]
    else:
        df_sql[dest_col] = None

# ----------------------------------------------------------------------
# 5.  Ajustes de tipos
# ----------------------------------------------------------------------
numeric_cols = ["cantidad", "precio_s_impuestos", "impuesto", "subtotal", "descuento",
                "importe_c_desc", "total_venta", "monto_pagado", "precio_bruto_s_desc"]
for c in numeric_cols:
    df_sql[c] = pd.to_numeric(df_sql[c], errors="coerce")

df_sql["hora_round"] = pd.to_datetime(df_sql["hora"], errors="coerce").dt.floor("S").dt.time

# ----------------------------------------------------------------------
# 6.  Trunca columnas que exceden longitud de la tabla destino
# ----------------------------------------------------------------------
def obtener_limites(cursor, table_name="stage_napse", schema="dbo"):
    qry = f"""
    SELECT COLUMN_NAME, CHARACTER_MAXIMUM_LENGTH
    FROM INFORMATION_SCHEMA.COLUMNS
    WHERE TABLE_SCHEMA = '{schema}' AND TABLE_NAME = '{table_name}'
          AND CHARACTER_MAXIMUM_LENGTH IS NOT NULL
    """
    cursor.execute(qry)
    return {row.COLUMN_NAME: row.CHARACTER_MAXIMUM_LENGTH for row in cursor.fetchall()}

with pyodbc.connect(
    f"DRIVER={{{SQL_DRIVER}}};SERVER={SQL_SERVER};DATABASE={SQL_DB};UID={SQL_USER};PWD={SQL_PASS};Encrypt=yes;TrustServerCertificate=yes"
) as cnxn:
    schema_limits = obtener_limites(cnxn.cursor())

for col, max_len in schema_limits.items():
    if col in df_sql.columns and df_sql[col].notna().any():
        mask = df_sql[col].astype(str).str.len() > max_len
        if mask.any():
            print(f"⚠️  Truncando columna '{col}' → {mask.sum()} filas excedían {max_len} caracteres.")
            df_sql[col] = df_sql[col].astype(str).str.slice(0, max_len)

# ----------------------------------------------------------------------
# 7.  Inserta en SQL Server
# ----------------------------------------------------------------------
with pyodbc.connect(
    f"DRIVER={{{SQL_DRIVER}}};SERVER={SQL_SERVER};DATABASE={SQL_DB};UID={SQL_USER};PWD={SQL_PASS};Encrypt=yes;TrustServerCertificate=yes"
) as cnxn:
    cursor = cnxn.cursor()
    cursor.fast_executemany = True

    placeholders = ", ".join("?" * len(ordered_cols))
    insert_sql = f"INSERT INTO {TABLE_NAME} ({', '.join(ordered_cols)}) VALUES ({placeholders})"

    data = [tuple(r) for r in df_sql.itertuples(index=False, name=None)]

    for i in range(0, len(data), BATCH_SIZE):
        chunk = data[i:i + BATCH_SIZE]
        cursor.executemany(insert_sql, chunk)
        cnxn.commit()
        print(f"✅  Insertadas {i + len(chunk):,}/{len(data):,} filas")

print("🎉  Carga completada sin errores")

try:
    DESTINO_OK.mkdir(parents=True, exist_ok=True)  # Crea la carpeta si no existe
    destino_final = DESTINO_OK / FILE_PATH.name
    shutil.move(str(FILE_PATH), str(destino_final)) 
    print(f"📦 Archivo movido a: {DESTINO_OK / FILE_PATH.name}")
except Exception as e:
    print(f"⚠️  No se pudo mover el archivo: {e}")


🧹 Eliminando registros de stage_napse entre 20250615 y 20250625...
✅ Registros eliminados exitosamente


C:\Users\jmarquez\AppData\Local\Temp\ipykernel_17376\1110630609.py:154: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_sql["hora_round"] = pd.to_datetime(df_sql["hora"], errors="coerce").dt.floor("S").dt.time
C:\Users\jmarquez\AppData\Local\Temp\ipykernel_17376\1110630609.py:154: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  df_sql["hora_round"] = pd.to_datetime(df_sql["hora"], errors="coerce").dt.floor("S").dt.time


⚠️  Truncando columna 'hora_round' → 5312 filas excedían 7 caracteres.
✅  Insertadas 5,000/5,312 filas
✅  Insertadas 5,312/5,312 filas
🎉  Carga completada sin errores
📦 Archivo movido a: \\192.168.10.5\ETLs_input\BI\NAPSE\Ventas\Insertados_reingenieria\Insertados_reingenieria\SalesNapse_2025-06-26.csv
